In [1]:
# import 
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook, show as bokeh_show # Import show as bokeh_show to avoid conflict
from bokeh.models import NumeralTickFormatter, CustomJS, Slider
from bokeh.layouts import column, row
from bokeh.palettes import Category10 # Import Category10 palette
import time
from IPython.display import display, HTML, clear_output

# Configure Bokeh to output plots directly in the notebook
output_notebook()

Loading BokehJS ...

In [2]:
# Load Data
df = pd.read_csv('dataset.csv')
print("Dataset loaded successfully.")
print(f"Initial dataset shape: {df.shape}")
print("Dataset head:")
display(df.head())

Dataset loaded successfully.
Initial dataset shape: (18368, 12)
Dataset head:


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [3]:
# Ensure 'Capacity' is not zero to avoid division by zero
df['Capacity'] = df['Capacity'].replace(0, 1)

# Map categorical 'VehicleType' to numerical weights
vehicle_type_weights = {
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5,
    'cycle': 0.2
}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_type_weights)
df['VehicleTypeWeight'].fillna(df['VehicleTypeWeight'].mean(), inplace=True)


# Map categorical 'TrafficConditionNearby' to numerical values
traffic_condition_mapping = {
    'low': 0.2,
    'average': 0.6,
    'high': 1.0
}
df['TrafficConditionNumeric'] = df['TrafficConditionNearby'].map(traffic_condition_mapping)
df['TrafficConditionNumeric'].fillna(df['TrafficConditionNumeric'].mean(), inplace=True)


# Initialize a base price for each unique parking lot (SystemCodeNumber)
unique_parking_lots = df['SystemCodeNumber'].unique()
base_prices = {lot: 5.0 for lot in unique_parking_lots} # Example base price: $5.00

# Create a DataFrame to store real-time pricing and demand data
realtime_data = df.groupby('SystemCodeNumber').first().reset_index()
realtime_data['current_price_model1'] = realtime_data['SystemCodeNumber'].map(base_prices)
realtime_data['current_price_model2'] = realtime_data['SystemCodeNumber'].map(base_prices)
realtime_data['current_price_model3'] = realtime_data['SystemCodeNumber'].map(base_prices)
realtime_data['demand_model2'] = 0.0 # Initialize demand for Model 2
realtime_data['competitor_price_sim'] = 0.0 # Initialize simulated competitor price

print("\nPre-processing complete. Initialized base prices and real-time data structure.")
print("Real-time data head:")
display(realtime_data.head())



Pre-processing complete. Initialized base prices and real-time data structure.
Real-time data head:


C:\Users\ankit\AppData\Local\Temp\ipykernel_3708\2982151243.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['VehicleTypeWeight'].fillna(df['VehicleTypeWeight'].mean(), inplace=True)
C:\Users\ankit\AppData\Local\Temp\ipykernel_3708\2982151243.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

,SystemCodeNumber,ID,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,VehicleTypeWeight,TrafficConditionNumeric,current_price_model1,current_price_model2,current_price_model3,demand_model2,competitor_price_sim
0,BHMBCCMKT01,0,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,1.0,0.2,5.0,5.0,5.0,0.0,0.0
1,BHMBCCTHL01,1312,387,26.144495,91.736205,120,car,low,2,0,04-10-2016,07:59:00,1.0,0.2,5.0,5.0,5.0,0.0,0.0
2,BHMEURBRD01,2624,470,26.149020,91.739503,117,car,low,2,0,04-10-2016,07:59:00,1.0,0.2,5.0,5.0,5.0,0.0,0.0
3,BHMMBMMBX01,3936,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,1.0,0.2,5.0,5.0,5.0,0.0,0.0
4,BHMNCPHST01,5248,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,0.5,0.2,5.0,5.0,5.0,0.0,0.0


In [4]:
# Helper Functions
def apply_price_bounds(price, base_price, min_factor=0.5, max_factor=2.0):
    """
    Applies lower and upper bounds to the calculated price based on a base price.
    Prices cannot go below min_factor * base_price or above max_factor * base_price.
    """
    lower_bound = base_price * min_factor
    upper_bound = base_price * max_factor
    return np.clip(price, lower_bound, upper_bound)

def normalize_value(value, min_val, max_val):
    """
    Normalizes a value to a range between 0 and 1.
    Handles cases where min_val equals max_val to prevent division by zero.
    """
    if max_val == min_val:
        return 0.0
    return (value - min_val) / (max_val - min_val)

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on Earth using the Haversine formula.
    """
    R = 6371 # Radius of Earth in kilometers
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance

print("Helper functions defined.")

Helper functions defined.


##  Pricing Models Implementation

In [5]:
### Model 1: Baseline Linear Approach
def calculate_price_model1(current_price, occupancy, capacity, alpha=2.0):
    """
    Calculates the new price using the Baseline Linear Model.
    Price_t+1 = Price_t + alpha * (Occupancy / Capacity)
    """
    occupancy_rate = occupancy / capacity
    new_price = current_price + alpha * occupancy_rate
    return new_price

print("Model 1 function defined.")

Model 1 function defined.


In [6]:
#Model 2: Demand-Responsive Pricing Function
# Define coefficients for Model 2's demand function.
MODEL2_COEFFICIENTS = {
    'occupancy_rate': 5.0,
    'queue_length': 3.0,
    'traffic_numeric': 2.0,
    'is_special_day': 4.0,
    'vehicle_type_weight': 1.0
}

# Sensitivity coefficient for how aggressively demand influences price
MODEL2_LAMBDA = 0.5

def calculate_demand_model2(row_data):
    """
    Calculates a composite demand score based on various factors.
    """
    occupancy_rate = row_data['Occupancy'] / row_data['Capacity']
    queue_length = row_data['QueueLength']
    traffic_numeric = row_data['TrafficConditionNumeric']
    is_special_day = row_data['IsSpecialDay']
    vehicle_type_weight = row_data['VehicleTypeWeight']

    demand = (
        MODEL2_COEFFICIENTS['occupancy_rate'] * occupancy_rate +
        MODEL2_COEFFICIENTS['queue_length'] * queue_length +
        MODEL2_COEFFICIENTS['traffic_numeric'] * traffic_numeric +
        MODEL2_COEFFICIENTS['is_special_day'] * is_special_day +
        MODEL2_COEFFICIENTS['vehicle_type_weight'] * vehicle_type_weight
    )
    return demand

def calculate_price_model2(base_price, demand, lambda_coeff=MODEL2_LAMBDA):
    """
    Calculates the new price using the Demand-Responsive Model.
    Price = Base Price * (1 + lambda * NormalizedDemand)
    """
    min_demand_val = 0.4
    max_demand_val = 42.5

    normalized_demand = normalize_value(demand, min_demand_val, max_demand_val)
    normalized_demand = max(0, normalized_demand)

    new_price = base_price * (1 + lambda_coeff * normalized_demand)
    return new_price

print("Model 2 functions defined.")


Model 2 functions defined.


In [7]:
# Model 3: Competitive Pricing (Conceptual)
def calculate_price_model3(current_lot_data, all_parking_data, base_price, proximity_threshold_km=0.5):
    """
    Calculates the new price considering competitive factors.
    """
    current_lat = current_lot_data['Latitude']
    current_lon = current_lot_data['Longitude']
    current_occupancy = current_lot_data['Occupancy']
    current_capacity = current_lot_data['Capacity']

    initial_price = current_lot_data['current_price_model2']

    competitors = []
    for idx, other_lot in all_parking_data.iterrows():
        if other_lot['SystemCodeNumber'] != current_lot_data['SystemCodeNumber']:
            distance = haversine_distance(current_lat, current_lon,
                                          other_lot['Latitude'], other_lot['Longitude'])
            if distance <= proximity_threshold_km:
                competitors.append(other_lot)

    simulated_competitor_price = initial_price

    if competitors:
        competitor_prices = []
        for comp_lot in competitors:
            comp_occupancy_rate = comp_lot['Occupancy'] / comp_lot['Capacity']
            comp_price = base_price * (1 + comp_occupancy_rate * 0.8 + np.random.uniform(-0.1, 0.1))
            competitor_prices.append(comp_price)

        if competitor_prices:
            simulated_competitor_price = np.mean(competitor_prices)

            if (current_occupancy / current_capacity) > 0.9 and simulated_competitor_price < initial_price:
                new_price = max(base_price * 0.8, simulated_competitor_price + (initial_price - simulated_competitor_price) * 0.2)
                return apply_price_bounds(new_price, base_price), simulated_competitor_price

            if simulated_competitor_price > initial_price:
                new_price = initial_price + (simulated_competitor_price - initial_price) * 0.5
                return apply_price_bounds(new_price, base_price), simulated_competitor_price

    return apply_price_bounds(initial_price, base_price), simulated_competitor_price

print("Model 3 function defined.")

Model 3 function defined.


In [ ]:
## 5. Real-time Simulation and Visualization with Bokeh
plots_to_show = list(unique_parking_lots[:5])

# Ensure there are parking lots to show
if not plots_to_show:
    print("No unique parking lots found to display. Please check your dataset.")
else:
    # Get a palette of colors from Category10 for the number of plots to show
    num_colors_needed = max(len(plots_to_show), 4)
    colors_palette = Category10[min(num_colors_needed, 10)]

    # Initialize ColumnDataSource for each plot
    sources = {}
    for lot_id in plots_to_show:
        sources[lot_id] = ColumnDataSource(data={
            'time': [],
            'price_model1': [],
            'price_model2': [],
            'price_model3': [],
            'competitor_price': []
        })

    # Create Bokeh plots
    plots = []
    for i, lot_id in enumerate(plots_to_show):
        p = figure(
            height=300,
            width=800,
            title=f"Dynamic Pricing for Parking Lot: {lot_id}",
            x_axis_label="Time Step",
            y_axis_label="Price ($)",
            tools="pan,wheel_zoom,box_zoom,reset,save",
            sizing_mode="scale_width"
        )
        p.line(x='time', y='price_model1', source=sources[lot_id], legend_label="Model 1 (Baseline)", line_width=2, color=colors_palette[0])
        p.line(x='time', y='price_model2', source=sources[lot_id], legend_label="Model 2 (Demand-Responsive)", line_width=2, color=colors_palette[1])
        p.line(x='time', y='price_model3', source=sources[lot_id], legend_label="Model 3 (Competitive)", line_width=2, color=colors_palette[2])
        p.line(x='time', y='competitor_price', source=sources[lot_id], legend_label="Simulated Competitor Price", line_width=2, color=colors_palette[3], line_dash='dashed')

        p.yaxis.formatter = NumeralTickFormatter(format="$0.00")
        p.legend.location = "top_left"
        p.legend.click_policy="hide"
        plots.append(p)

    # Arrange plots in a column layout
    layout = column(plots, sizing_mode="scale_width")

    # %%
    # Simulation Loop Parameters
    simulation_steps = len(df)
    time_interval_seconds = 0.1

    # Store historical prices for each lot for plotting
    historical_prices = {lot: {'model1': [], 'model2': [], 'model3': [], 'competitor': []} for lot in unique_parking_lots}
    historical_time_steps = {lot: [] for lot in unique_parking_lots}

    print("Starting real-time pricing simulation...")
    print("Plots will update below. Please wait for the simulation to run.")

    current_time_step = 0

    # Group the DataFrame by 'SystemCodeNumber' to iterate through each lot's data sequentially
    # Corrected format string for pd.to_datetime
    df['DateTime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format="%d-%m-%Y %H:%M:%S")
    df_sorted = df.sort_values(by='DateTime').reset_index(drop=True)

    # Iterate through the dataset row by row to simulate real-time updates
    for index, row_data in df_sorted.iterrows():
        lot_id = row_data['SystemCodeNumber']
        base_price = base_prices[lot_id]

        # --- Model 1 Calculation ---
        new_price_model1 = calculate_price_model1(
            realtime_data.loc[realtime_data['SystemCodeNumber'] == lot_id, 'current_price_model1'].iloc[0],
            row_data['Occupancy'],
            row_data['Capacity']
        )
        new_price_model1 = apply_price_bounds(new_price_model1, base_price)
        realtime_data.loc[realtime_data['SystemCodeNumber'] == lot_id, 'current_price_model1'] = new_price_model1

        # --- Model 2 Calculation ---
        demand_m2 = calculate_demand_model2(row_data)
        realtime_data.loc[realtime_data['SystemCodeNumber'] == lot_id, 'demand_model2'] = demand_m2
        new_price_model2 = calculate_price_model2(base_price, demand_m2)
        new_price_model2 = apply_price_bounds(new_price_model2, base_price)
        realtime_data.loc[realtime_data['SystemCodeNumber'] == lot_id, 'current_price_model2'] = new_price_model2

        # --- Model 3 Calculation ---
        realtime_data_copy_for_model3 = realtime_data.copy()
        new_price_model3, simulated_competitor_price = calculate_price_model3(
            realtime_data_copy_for_model3.loc[realtime_data_copy_for_model3['SystemCodeNumber'] == lot_id].iloc[0],
            realtime_data_copy_for_model3,
            base_price
        )
        new_price_model3 = apply_price_bounds(new_price_model3, base_price)
        realtime_data.loc[realtime_data['SystemCodeNumber'] == lot_id, 'current_price_model3'] = new_price_model3
        realtime_data.loc[realtime_data['SystemCodeNumber'] == lot_id, 'competitor_price_sim'] = simulated_competitor_price


        # Update historical data for plotting
        historical_time_steps[lot_id].append(current_time_step)
        historical_prices[lot_id]['model1'].append(new_price_model1)
        historical_prices[lot_id]['model2'].append(new_price_model2)
        historical_prices[lot_id]['model3'].append(new_price_model3)
        historical_prices[lot_id]['competitor'].append(simulated_competitor_price)

        # Update Bokeh plots for selected lots
        if lot_id in plots_to_show:
            new_data = {
                'time': historical_time_steps[lot_id],
                'price_model1': historical_prices[lot_id]['model1'],
                'price_model2': historical_prices[lot_id]['model2'],
                'price_model3': historical_prices[lot_id]['model3'],
                'competitor_price': historical_prices[lot_id]['competitor']
            }
            sources[lot_id].data = new_data

            # Clear previous output and display the updated plot
            clear_output(wait=True)
            bokeh_show(layout) # No need for notebook_handle here as we are re-rendering

        current_time_step += 1
        time.sleep(time_interval_seconds)

    print("\nSimulation complete.")